In [1]:
"""

LICENSE MIT
2020
Guillaume Rozier
Website : http://www.guillaumerozier.fr
Mail : guillaume.rozier@telecomnancy.net

README:
This file contains a script that automatically update data. In the morning it update World data, and it updates French data as soon as they are released by Santé publique France.
"""

'\n\nLICENSE MIT\n2020\nGuillaume Rozier\nWebsite : http://www.guillaumerozier.fr\nMail : guillaume.rozier@telecomnancy.net\n\nThis file contains a script that automatically update data. In the morning it update World data, and it updates French data as soon as they are released by Santé publique France.\n'

In [56]:
import datetime
import time
import subprocess
import requests
import re

### FUNCTION DEFINITIONS ###
url_metadata = "https://www.data.gouv.fr/fr/organizations/sante-publique-france/datasets-resources.csv"
metadata = requests.get(url_metadata)
content = str(metadata.content)

def update_repo():
    subprocess.run(["sudo", "git", "fetch", "--all"])
    subprocess.run(["sudo", "git", "reset", "--hard", "origin/master"])
    subprocess.run(["sudo", "jupyter", "nbconvert", "--to", "script", "*.ipynb"])
    
def push(type_data):
    subprocess.run(["sudo", "git", "add", "images/", "data/"])
    subprocess.run(["sudo", "git", "commit", "-m", "[auto] data update: {}".format(type_data)])
    subprocess.run(["git", "push"])
    print("pushed")
    
def try_update_france(time_re):
    data_today = False
    metadata = requests.get(url_metadata)
    content = str(metadata.content)
    
    if ("donnees-hospitalieres-covid19-2020-{}-{}".format(x.strftime("%m"), x.strftime("%d")) in content):
        print("starting France update: {}:{}".format(str(now.hour), str(now.minute)))
        update_repo()
        
        re_result = re.search("donnees-hospitalieres-nouveaux-covid19-2020-[0-9]{2}-[0-9]{2}-[0-9]{2}h[0-9]{2}.csv", content)
        time_re = re.match(".*([0-9]{2})h([0-9]{2}).csv", re_result[0])

        # Si la data actuelle correspond à celle du fichier
        fr_update = datetime.datetime.now() # Alors le fichier est à jour

        # Mise à jour des graphiques
        subprocess.run(["sudo", "python3", "covid19_france_charts.py"])
        push("France")
        print("update France: " + str(now.hour) + ":" + str(now.minute))

        subprocess.run(["sudo", "python3", "covid19_france_maps.py"])
        push("France GIF")
        print("update France GIF: " + str(now.hour) + ":" + str(now.minute))
    return fr_update, time_re

    
### MAIN FUNCTION ###
x = datetime.datetime.now()
fr_update = x
world_update = x
c = 0
k = 1

re_result = re.search("donnees-hospitalieres-nouveaux-covid19-[0-9]{4}-[0-9]{2}-[0-9]{2}-[0-9]{2}h[0-9]{2}.csv", content)
re_date = re.match(".*2020-([0-9]{2})-([0-9]{2})-([0-9]{2})h([0-9]{2}).csv", re_result[0])
last_update_france = {'month' = re_date[1], 'day' = re_date[2], 'hour' = re_date[3], 'minute' = re_date[4]}
current_update_france = last_update_france

print("will enter loop")
while True:
    
    now = datetime.datetime.now() 
    
    metadata = requests.get(url_metadata)
    content = str(metadata.content)
    re_result = re.search("donnees-hospitalieres-nouveaux-covid19-[0-9]{4}-[0-9]{2}-[0-9]{2}-[0-9]{2}h[0-9]{2}.csv", content)
    time_re = re.match(".*([0-9]{2})h([0-9]{2}).csv", re_result[0])
    
    if (now.hour >= 4) & (now.hour <= 7) & (world_update.day != now.day):
        print("starting world update: {}:{}".format(str(now.hour), str(now.minute)))
        # S'il est 05h05
        
        world_update = now
        
        # mise à jour des graphiques world
        update_repo()
        subprocess.run(["sudo", "python3", "covid19_world_charts.py"])
        push("World")
        print("update World pushed: " + str(now.hour) + ":" + str(now.minute))
        
        # ... et France (certains en dépendent)
        subprocess.run(["sudo", "python3", "covid19_france_charts.py"])
        push("France")
        print("update France pushed: " + str(now.hour) + ":" + str(now.minute))
        time.sleep(30)
        
            

    if ( (((now.hour == 18) & (now.minute >= 59)) or ((now.hour >= 19) & (now.hour<= 20))) ):
        while ( (((now.hour == 18) & (now.minute >= 59)) or ((now.hour >= 19) & (now.hour<= 20))) & (last_update_france ) ):
            # Si l'heure comprise entre 18h59 et 21h59, ET les données PAS à jour depuis plus de 2h30
            fr_update, last_time_re = try_update_france(last_time_re)
            time.sleep(20)
            
    else: # S'il n'est pas entre 18h59 et 21h59
        if( (now - fr_update).seconds/3600 > 24): # S'il s'est écoulé plus de 24h depuis la dernière update
            if (k%30 == 0): #Check 1 fois toutes les 20 sec * 30 (environ 10 min)
                fr_update = try_update_france()
            k += 1
            
    time.sleep(30)

will enter loop


KeyboardInterrupt: 

In [89]:
import re
url_metadata = "https://www.data.gouv.fr/fr/organizations/sante-publique-france/datasets-resources.csv"
metadata = requests.get(url_metadata)
txt = metadata.content
re_date = re.match(".*2020-([0-9]{2})-([0-9]{2})-([0-9]{2})h([0-9]{2}).csv", str(txt))

In [95]:
re_date[4]

'47'

In [85]:


m[2]

'47'

In [55]:
x + datetime.timedelta(5000)

datetime.datetime(2033, 12, 24, 15, 16, 41, 950408)